In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn import metrics
import os

# Modelo DNN

In [3]:
# Transformaciones (normalización simple a [0,1])
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # asegura canal único
    transforms.ToTensor()
])

In [4]:
# Carga dataset desde la carpeta raíz generada por TINTOlib
ruta_carpeta_actual = os.getcwd()
data_dir  = os.path.join(ruta_carpeta_actual, "PuntosMuestra_CR_tinto_synthetic_images")
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split train/test
n = len(dataset)
n_train = int(0.7*n)
train_ds, test_ds = torch.utils.data.random_split(dataset, [n_train, n-n_train])

In [5]:
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl  = DataLoader(test_ds, batch_size=64)

In [14]:
import torch, torch.nn as nn

class SmallCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.head = None  # se define luego

    def build_head(self, x_sample, num_classes):
        with torch.no_grad():
            f = self.conv(x_sample).flatten(1).shape[1]
        self.head = nn.Sequential(
            nn.Linear(f,128), nn.ReLU(),
            nn.Linear(128,num_classes)
        )

    def forward(self, x):
        z = self.conv(x).flatten(1)
        return self.head(z)

In [15]:
x,y = next(iter(train_dl))
print("Batch shape:", x.shape)  # [B,1,H,W]
print("Num clases:", len(dataset.classes))

model = SmallCNN(num_classes=len(dataset.classes))
x,_ = next(iter(train_dl))
model.build_head(x, num_classes=len(dataset.classes))

Batch shape: torch.Size([32, 1, 20, 20])
Num clases: 7


# Entrenamiento

In [16]:
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    for x,y in train_dl:
        opt.zero_grad()
        loss = crit(model(x), y)
        loss.backward()
        opt.step()

    # Validación
    model.eval()
    correct=total=0
    with torch.no_grad():
        for x,y in test_dl:
            pred = model(x).argmax(1)
            correct += (pred==y).sum().item()
            total += y.numel()
    print(f"Epoch {epoch+1}: Acc={correct/total:.3f}")


Epoch 1: Acc=0.652
Epoch 2: Acc=0.814
Epoch 3: Acc=0.825
Epoch 4: Acc=0.824
Epoch 5: Acc=0.822
Epoch 6: Acc=0.827
Epoch 7: Acc=0.824
Epoch 8: Acc=0.828
Epoch 9: Acc=0.831
Epoch 10: Acc=0.827
